# Credit Card Spending Prediction

A credit card company has provided historical data on its active customers. The data includes demographic and behavioral information on customers. Your task is to build a model to predict monthly credit card spend for individual customers.

##### Initially started with Exploratory Data Analysis of the data. and Uncovered the distributions and relationships between the variables.

In [ ]:
# Scatterplots with Monthly_spend
for i in list(data.select_dtypes('number').columns):
    sns.scatterplot(data=data,x=i,y='monthly_spend')
    plt.show()

# Histograms
for i in list(data.select_dtypes('number').columns):
    sns.histplot(data=data,x=i)
    plt.show()

Used Ydata profiling to get an understanding as a part of EDA

In [ ]:
from ydata_profiling import ProfileReport
report=ProfileReport(df=eda)
report

# Submission 1
Initially started with a Linear Regression Model with variables that were not having any missing values.

In [ ]:
X_train = data.drop(labels=['monthly_spend','education_level','online_shopping_freq','utility_payment_count'],axis=1)
y = data.monthly_spend
categorical_variables=['gender','marital_status','region','employment_status','card_type']
X= pd.get_dummies(X_train, 
                         prefix_sep = '_', 
                         columns = categorical_variables, 
                         drop_first = True)

from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X,y)
reg.predict(X)


# PREDICTION
scoring_data = pd.read_csv('Datasets/scoring_data.csv')
scoring_data.drop(labels=['education_level','online_shopping_freq','utility_payment_count'],axis=1,inplace=True)
X_test = pd.get_dummies(scoring_data, 
                         prefix_sep = '_', 
                         columns = categorical_variables, 
                         drop_first = True)


pred = reg.predict(X_test)

# Submission 2
Then Moved ahead with performing Linear Regression mmodel with imputing mode for categorical missing variables and median for quantitative variables having missing values as the data was right skewed.

In [ ]:
data['education_level'].fillna(data['education_level'].mode()[0],inplace=True)
data['online_shopping_freq'].fillna(data['online_shopping_freq'].median(),inplace=True)
data['utility_payment_count'].fillna(data['utility_payment_count'].median(),inplace=True)

X_train = data.drop(labels='monthly_spend',axis=1)
y = data.monthly_spend
categorical_variables=['gender','marital_status','education_level','region','employment_status','card_type']
X= pd.get_dummies(X_train, 
                         prefix_sep = '_', 
                         columns = categorical_variables, 
                         drop_first = True)

from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X,y)
reg.predict(X)

# PREDICTION
scoring_data = pd.read_csv('Datasets/scoring_data.csv')
# Imputation


scoring_data['education_level'].fillna(data['education_level'].mode()[0],inplace=True)
scoring_data['online_shopping_freq'].fillna(data['online_shopping_freq'].median(),inplace=True)
scoring_data['utility_payment_count'].fillna(data['utility_payment_count'].median(),inplace=True)

X_test = pd.get_dummies(scoring_data, 
                         prefix_sep = '_', 
                         columns = categorical_variables, 
                         drop_first = True)

pred = reg.predict(X_test)

# Submission 3
Moving Ahead with submission 3. I used best subset selection method  using same imputations mode and medians and fitted a multiple linear regression model

In [ ]:
# Imputation
data['education_level'].fillna(data['education_level'].mode()[0],inplace=True)
data['online_shopping_freq'].fillna(data['online_shopping_freq'].median(),inplace=True)
data['utility_payment_count'].fillna(data['utility_payment_count'].median(),inplace=True)

# Splitting data and getting dummies
X_train = data.drop(labels='monthly_spend',axis=1)
y = data.monthly_spend
categorical_variables=['gender','marital_status','education_level','region','employment_status','card_type']
X= pd.get_dummies(X_train, 
                         prefix_sep = '_', 
                         columns = categorical_variables, 
                         drop_first = True)

# Forward Selection
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
sfs = SequentialFeatureSelector(LinearRegression(),
                                  k_features='best',       
                                  forward=True,
                                  floating=False,          
                                  scoring='r2',
                                  cv=5)
sfs = sfs.fit(X, y)
pd.DataFrame(sfs.get_metric_dict()).T.loc[:,['feature_names','avg_score']].sort_values('avg_score', ascending = False)



# Backward Elimination
bfs = SequentialFeatureSelector(LinearRegression(),
          k_features='best',       
          forward=False,
          floating=False,          
          scoring='r2',
          cv=5)
bfs = bfs.fit(X, y)
pd.DataFrame(sfs.get_metric_dict()).T.loc[:,['feature_names','avg_score']].sort_values('avg_score', ascending = False)

# Stepwise

stfs = SequentialFeatureSelector(LinearRegression(),
          k_features='best',       
          forward=True,
          floating=True,           
          scoring='r2',
          cv=5)
stfs = stfs.fit(X, y)
pd.DataFrame(sfs.get_metric_dict()).T.loc[:,['feature_names','avg_score']].sort_values('avg_score', ascending = False)

# All variations give same results

variables=bfs.k_feature_names_
variables=list(variables)
variables

# Fitting Regression model

reg=LinearRegression()
reg.fit(X[variables],y)
reg.predict(X[variables])

# PREDICTION
scoring_data = pd.read_csv('Datasets/scoring_data.csv')
# Imputation


scoring_data['education_level'].fillna(data['education_level'].mode()[0],inplace=True)
scoring_data['online_shopping_freq'].fillna(data['online_shopping_freq'].median(),inplace=True)
scoring_data['utility_payment_count'].fillna(data['utility_payment_count'].median(),inplace=True)

X_test = pd.get_dummies(scoring_data, 
                         prefix_sep = '_', 
                         columns = categorical_variables, 
                         drop_first = True)

pred = reg.predict(X_test[variables])

# Submission 4
In this submission I build on the submission 3 backward selection method. I used all variables that were used for submission 3. Then for numeric variables I perform EDA and then look for anytransformations.
Majority of the variables were right skewed hence I applied log transformation to the right skewed variables.
Basic EDA for categorical variables was done but we use them as they are from the submission 3.
RMSE=277.70931827231027

In [ ]:
`missing_columns=['education_level','online_shopping_freq','utility_payment_count']

eda=data[missing_columns]

data['education_level'].fillna(data['education_level'].mode()[0],inplace=True)
data['online_shopping_freq'].fillna(data['online_shopping_freq'].median(),inplace=True)
`data['utility_payment_count'].fillna(data['utility_payment_count'].median(),inplace=True)

data['credit_limit']=np.log(data['credit_limit'])
data['reward_points_balance']=np.log(data['reward_points_balance'])
data['num_transactions']=np.log(data['num_transactions']+1)
data['avg_transaction_value']=np.log(data['avg_transaction_value'])
data['travel_frequency']=np.log(data['travel_frequency']+1)
data['online_shopping_freq']=np.log(data['online_shopping_freq']+1)
data['utility_payment_count']=np.log(data['utility_payment_count']+1)

variables=['owns_home',
 'has_auto_loan',
 'credit_score',
 'credit_limit',
 'tenure',
 'num_transactions',
 'avg_transaction_value',
 'online_shopping_freq',
 'reward_points_balance',
 'travel_frequency',
 'utility_payment_count',
 'num_children',
 'num_credit_cards',
 'gender',
 'marital_status',
 'education_level',
 'region',
 'employment_status',
 'card_type']

 X_train = data[variables]
y = data.monthly_spend
categorical_variables=['gender','marital_status','education_level','region','employment_status','card_type']
X= pd.get_dummies(X_train, 
                         prefix_sep = '_', 
                         columns = categorical_variables, 
                         drop_first = True)

from sklearn.linear_model import LinearRegression

model=LinearRegression()
model.fit(X,y)
model.predict(X)

# PREDICTION
s_data = pd.read_csv('Datasets/scoring_data.csv')

variables=['owns_home',
 'has_auto_loan',
 'credit_score',
 'credit_limit',
 'tenure',
 'num_transactions',
 'avg_transaction_value',
 'online_shopping_freq',
 'reward_points_balance',
 'travel_frequency',
 'utility_payment_count',
 'num_children',
 'num_credit_cards',
 'gender',
 'marital_status',
 'education_level',
 'region',
 'employment_status',
 'card_type']

# Imputation
scoring_data=s_data[variables]

scoring_data['education_level'].fillna(data['education_level'].mode()[0],inplace=True)
scoring_data['online_shopping_freq'].fillna(data['online_shopping_freq'].median(),inplace=True)
scoring_data['utility_payment_count'].fillna(data['utility_payment_count'].median(),inplace=True)


# Submission 5
I tried the MICE algorithm to fill in the missing values and then test it based on the best working submission 3. Train Test split was used and data was trained using the train dataset.
Resulted in Lower test rmse score but not performed well in the actual scoring dataset.

In [ ]:
y=data['monthly_spend']
X=data.drop(labels=['monthly_spend'],axis=1)

# Apply train test split
from sklearn.model_selection import train_test_split

X_train_raw,X_test_raw,y_train,y_test=train_test_split(X,y,train_size=0.8,random_state=2121)

umeric_variables=X_train_raw.select_dtypes('number').columns
categorical_columns = X_train_raw.select_dtypes(include=['object', 'category']).columns.tolist()

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns)
    ],
    remainder='passthrough' # Keep numeric columns as they are
)
X_train_raw_encoded=preprocessor.fit_transform(X_train_raw)
feature_names = list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns))
numeric_cols = X_train_raw.select_dtypes(include=['number']).columns.tolist()
new_cols = feature_names + numeric_cols

X_train_processed=pd.DataFrame(X_train_raw_encoded, columns=new_cols)

estimator = RandomForestRegressor(n_estimators=10, random_state=2121)

# 3. Create the Iterative Imputer (MICE)
mice_imputer = IterativeImputer(
    estimator=estimator,
    max_iter=10,        # Number of imputation cycles (T)
    initial_strategy='mean', # Initial placeholder strategy
    random_state=2121
)

# 4. Fit the Imputer and Transform the Data
# The imputer automatically handles all columns, using each as a predictor for the others.
df_imputed_array = mice_imputer.fit_transform(X_train_processed)

# Convert the resulting NumPy array back to a Pandas DataFrame
df_imputed = pd.DataFrame(df_imputed_array, columns=X_train_processed.columns)

# Lets try Stepwise
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

stfs = SequentialFeatureSelector(LinearRegression(),
          k_features='best',       
          forward=True,
          floating=True,           
          scoring='r2',
          cv=5)
stfs = stfs.fit(df_imputed, y_train)
pd.DataFrame(stfs.get_metric_dict()).T.loc[:,['feature_names','avg_score']].sort_values('avg_score', ascending = False)

X_train=df_imputed[list(stfs.k_feature_names_)]

reg=LinearRegression().fit(X_train,y_train)

# COnvert TEst Types

numeric_variables=X_test_raw.select_dtypes('number').columns
categorical_columns = X_test_raw.select_dtypes(include=['object', 'category']).columns.tolist()

X_test_raw_encoded=preprocessor.transform(X_test_raw)
feature_names = list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns))
numeric_cols = X_test_raw.select_dtypes(include=['number']).columns.tolist()
new_cols = feature_names + numeric_cols

X_test_processed=pd.DataFrame(X_test_raw_encoded, columns=new_cols)

# Transform mice

X_test_imputed_array = mice_imputer.transform(X_test_processed)
X_test= pd.DataFrame(X_test_imputed_array, columns=X_test_processed.columns)
X_test=X_test[list(stfs.k_feature_names_)]

y_hat_test=reg.predict(X_test)

scoring_data=pd.read_csv('Datasets/scoring_data.csv')

numeric_variables=scoring_data.select_dtypes('number').columns
categorical_columns = scoring_data.select_dtypes(include=['object', 'category']).columns.tolist()

scoring_data_encoded=preprocessor.transform(scoring_data)
feature_names = list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns))
numeric_cols = scoring_data.select_dtypes(include=['number']).columns.tolist()
new_cols = feature_names + numeric_cols

scoring_data_processed=pd.DataFrame(scoring_data_encoded, columns=new_cols)

# Transform mice

scoring_data_imputed_array = mice_imputer.transform(scoring_data_processed)
scoring_data_pred= pd.DataFrame(scoring_data_imputed_array, columns=scoring_data_processed.columns)
scoring_data_pred=scoring_data_pred[list(stfs.k_feature_names_)]

# Submission 6
Use the same method in submission 5 and train on full train dataset.
It was able to reduce the score to the lowest of first 5.

In [ ]:
y=data['monthly_spend']
X=data.drop(labels=['monthly_spend'],axis=1)

numeric_variables=X.select_dtypes('number').columns
categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns)
    ],
    remainder='passthrough' # Keep numeric columns as they are
)
X_encoded=preprocessor.fit_transform(X)
feature_names = list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns))
numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
new_cols = feature_names + numeric_cols

X_processed=pd.DataFrame(X_encoded, columns=new_cols)

estimator = RandomForestRegressor(n_estimators=10, random_state=2121)

# 3. Create the Iterative Imputer (MICE)
mice_imputer = IterativeImputer(
    estimator=estimator,
    max_iter=10,        # Number of imputation cycles (T)
    initial_strategy='mean', # Initial placeholder strategy
    random_state=2121
)

# 4. Fit the Imputer and Transform the Data
# The imputer automatically handles all columns, using each as a predictor for the others.
df_imputed_array = mice_imputer.fit_transform(X_processed)

# Convert the resulting NumPy array back to a Pandas DataFrame
df_imputed = pd.DataFrame(df_imputed_array, columns=X_processed.columns)

from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

stfs = SequentialFeatureSelector(LinearRegression(),
          k_features='best',       
          forward=True,
          floating=True,           
          scoring='r2',
          cv=5)
stfs = stfs.fit(df_imputed, y)
pd.DataFrame(stfs.get_metric_dict()).T.loc[:,['feature_names','avg_score']].sort_values('avg_score', ascending = False)



X_train=df_imputed[list(stfs.k_feature_names_)]

reg=LinearRegression().fit(X_train,y)
y_hat=reg.predict(X_train)

from sklearn.metrics import root_mean_squared_error

root_mean_squared_error(y,y_hat)

scoring_data=pd.read_csv('Datasets/scoring_data.csv')

numeric_variables=scoring_data.select_dtypes('number').columns
categorical_columns = scoring_data.select_dtypes(include=['object', 'category']).columns.tolist()

scoring_data_encoded=preprocessor.transform(scoring_data)
feature_names = list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns))
numeric_cols = scoring_data.select_dtypes(include=['number']).columns.tolist()
new_cols = feature_names + numeric_cols

scoring_data_processed=pd.DataFrame(scoring_data_encoded, columns=new_cols)

# Transform mice

scoring_data_imputed_array = mice_imputer.transform(scoring_data_processed)
scoring_data_pred= pd.DataFrame(scoring_data_imputed_array, columns=scoring_data_processed.columns)
scoring_data_pred=scoring_data_pred[list(stfs.k_feature_names_)]

pred=reg.predict(scoring_data_pred)

# Submission 7 
I used the same technique as the one above but the only change was I used median in regression imputation instead of mean. It didn't reduced the score.

# Submission 8
STandardize variables before regression. Performed well but didn't beat the mean imputation with MICE in submission 6.

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

# Load data
data = pd.read_csv('Datasets/analysis_data.csv')

y = data['monthly_spend']
X = data.drop(columns=['monthly_spend'])

# Identify column types
categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_columns = X.select_dtypes(include=['number']).columns.tolist()

# 1. OneHotEncoding
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns)
    ],
    remainder='passthrough'
)

X_encoded = preprocessor.fit_transform(X)
encoded_cat_cols = list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns))
new_cols = encoded_cat_cols + numeric_columns
X_encoded_df = pd.DataFrame(X_encoded, columns=new_cols)

# 2. MICE Imputation
mice = IterativeImputer(
    estimator=RandomForestRegressor(n_estimators=20, random_state=2121),
    max_iter=10,
    initial_strategy='mean',
    random_state=2121
)

X_imputed = mice.fit_transform(X_encoded_df)
X_imputed_df = pd.DataFrame(X_imputed, columns=new_cols)

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector

# 3. Stepwise Feature Selection (on imputed data)
stfs = SequentialFeatureSelector(
    LinearRegression(),
    k_features='best',
    forward=True,
    floating=True,
    scoring='r2',
    cv=5
)
stfs = stfs.fit(X_imputed_df, y)

selected_features = list(stfs.k_feature_names_)
X_train = X_imputed_df[selected_features]

# 4. Scale selected features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# 5. Fit Linear Regression
reg = LinearRegression()
reg.fit(X_train_scaled, y)

# 2. Impute using TRAINED MICE
scoring_imputed = mice.transscoring_data = pd.read_csv('Datasets/scoring_data.csv')

# Same categorical columns as training
categorical_columns_scoring = scoring_data.select_dtypes(include=['object', 'category']).columns.tolist()

# 1. Encode using TRAINED preprocessor
scoring_encoded = preprocessor.transform(scoring_data)
numeric_cols_scoring = scoring_data.select_dtypes(include=['number']).columns.tolist()
new_cols_scoring = encoded_cat_cols + numeric_cols_scoring

scoring_encoded_df = pd.DataFrame(scoring_encoded, columns=new_cols_scoring)

form(scoring_encoded_df)
scoring_imputed_df = pd.DataFrame(scoring_imputed, columns=new_cols_scoring)

# 3. Select SAME stepwise features
scoring_selected = scoring_imputed_df[selected_features]

# 4. Scale with TRAINED scaler
scoring_scaled = scaler.transform(scoring_selected)

# 5. Predict
pred = reg.predict(scoring_scaled)


# Submission 9
Tried Ridge regression and mice imputation, worked well. Log transformed Y  then tried ridge worked poorly.

In [ ]:
import pandas as pd

data = pd.read_csv('Datasets/analysis_data.csv')
y=data['monthly_spend']
X=data.drop(labels=['monthly_spend'],axis=1)

numeric_variables=X.select_dtypes('number').columns
categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns)
    ],
    remainder='passthrough' # Keep numeric columns as they are
)
X_encoded=preprocessor.fit_transform(X)
feature_names = list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns))
numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
new_cols = feature_names + numeric_cols

X_processed=pd.DataFrame(X_encoded, columns=new_cols)

estimator = RandomForestRegressor(n_estimators=10, random_state=2121)

# 3. Create the Iterative Imputer (MICE)
mice_imputer = IterativeImputer(
    estimator=estimator,
    max_iter=10,        # Number of imputation cycles (T)
    initial_strategy='mean', # Initial placeholder strategy
    random_state=2121
)

# 4. Fit the Imputer and Transform the Data
# The imputer automatically handles all columns, using each as a predictor for the others.
df_imputed_array = mice_imputer.fit_transform(X_processed)

# Convert the resulting NumPy array back to a Pandas DataFrame
df_imputed = pd.DataFrame(df_imputed_array, columns=X_processed.columns)


from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler

X_train = df_imputed

# 3. Ridge Regression with automatic alpha selection
ridge = RidgeCV(alphas=[0.1, 1.0, 10.0, 50.0, 100.0], cv=5)
ridge.fit(X_train, y)

# 4. Predictions
y_hat = ridge.predict(X_train)

import numpy as np

X_train2 = df_imputed
y_train_log = np.log1p(y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# 3. Ridge Regression with automatic alpha selection
ridge2 = RidgeCV(alphas=[0.1, 1.0, 10.0, 50.0, 100.0], cv=5)
ridge2.fit(X_train_scaled, y_train_log)

# 4. Predictions
y_hat2 = np.expm1(ridge2.predict(X_train_scaled))

root_mean_squared_error(y,y_hat2)

# Submisssion 10

This was the best Model Tried to fit the polynomial terms and then penalize them using Elastic net regression. It was robust and always generated less RMSE for every sample.

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge

data = pd.read_csv("Datasets/analysis_data.csv")

y = data["monthly_spend"]
X = data.drop(columns=["monthly_spend"])

categorical_columns = X.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_columns = X.select_dtypes(include=['number']).columns.tolist()

# OneHot Encoding
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), categorical_columns)
    ],
    remainder="passthrough"
)

X_encoded = preprocessor.fit_transform(X)
encoded_cat_cols = preprocessor.named_transformers_["cat"].get_feature_names_out(categorical_columns)
new_cols = list(encoded_cat_cols) + numeric_columns

X_encoded_df = pd.DataFrame(X_encoded, columns=new_cols)

# MICE Imputation using BayesianRidge
mice = IterativeImputer(
    estimator=BayesianRidge(),
    max_iter=10,
    initial_strategy='median',
    random_state=42
)

X_imputed = mice.fit_transform(X_encoded_df)
X_imputed_df = pd.DataFrame(X_imputed, columns=new_cols)

from sklearn.preprocessing import PolynomialFeatures

# Polynomial features 
poly = PolynomialFeatures(
    degree=2,
    interaction_only=True,   
    include_bias=False
)

X_poly = poly.fit_transform(X_imputed_df)
poly_feature_names = poly.get_feature_names_out(X_imputed_df.columns)

from sklearn.linear_model import ElasticNetCV

enet = ElasticNetCV(
    l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9],
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10],
    cv=5,
    random_state=42,
    max_iter=5000
)

enet.fit(X_poly, y)

coef = enet.coef_
features = poly.get_feature_names_out(new_cols)

significant_vars = pd.DataFrame({
    "feature": features,
    "coef": coef
})

significant_vars = significant_vars[significant_vars["coef"] != 0]
significant_vars = significant_vars.sort_values("coef", ascending=False)

scoring = pd.read_csv("Datasets/scoring_data.csv")

# Apply same OneHot encoder
X_scoring_encoded = preprocessor.transform(scoring)
numeric_scoring_cols = scoring.select_dtypes(include=['number']).columns.tolist()
X_scoring_encoded_df = pd.DataFrame(X_scoring_encoded, columns=new_cols)

# MICE imputation
X_scoring_imputed = mice.transform(X_scoring_encoded_df)
X_scoring_imputed_df = pd.DataFrame(X_scoring_imputed, columns=new_cols)

# Polynomial features
X_scoring_poly = poly.transform(X_scoring_imputed_df)

# Predict
pred = enet.predict(X_scoring_poly)


# Submission 11
Tried Polynomial regression with Degree 3 with ELastic Net but didn't work as the RMSE increased.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Polynomial features (degree=2 interactions only)
poly = PolynomialFeatures(
    degree=3,
    interaction_only=True,   # avoids squared terms if you want
    include_bias=False
)

X_poly = poly.fit_transform(X_imputed_df)
poly_feature_names = poly.get_feature_names_out(X_imputed_df.columns)

from sklearn.linear_model import ElasticNetCV

# Elastic Net without scaling X or y
enet = ElasticNetCV(
    # l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9],
    # alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10],
    l1_ratio=[0.9],
    alphas=[10],
    cv=5,
    random_state=42,
    max_iter=5000
)

enet.fit(X_poly, y)

I also fitted many Machine Learning models but they always overfitted the data.


Github : https://github.com/AtharvTungatkar/Predict-Credit-Card-Spending-Predictive-Analytics-Competition